## Webscraping with Requests Pandas



## Scope and Objectives:

In this lab I will:

1. Use `Pandas` to scrape data from a webpage
2. Perform fundamental exploratory data analysis on data sets
3. Create the extract, transform, and load (ETL) data engineering pipeline
    1. Extract: 
        - Extract data from a webpage as a pandas dataframe
        - Extract data of interest and drop other data
    2. Transform:
        - Split the whole dataset into two dataset by grouping particular columns in each subset.
        - Use the `lambda` anonymous function to modify the content of a dataframe
        - Handle missing data
        - Perform datatype conversion from `str` to numeric types
    3. Load:
        - Load data into csv files and sqlite database's tables
4. Run the ETL pipeline and log the results
5. Access the database and execute basic and advanced SQL queries and visualizations.

In [1]:
# Import required packages
import pandas as pd
import sqlite3
from datetime import datetime
import requests

In [2]:
# The webpage to scrape
## The webpage contains several tables that contain poverty data per country
## This is a retrieved old version of the page before most recent updates
url='https://en.wikipedia.org/w/index.php?title=List_of_countries_by_percentage_of_population_living_in_poverty&oldid=1216916528'

In [3]:
# modify the request User-Agent Header to avoid being blocked by the server
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"
}

In [4]:
# Send an HTTP request (get method) to retrieve the whole website
r = requests.get(url=url, headers=headers)

# display the status code
print("Status code:", r.status_code)

Status code: 200


In [5]:
# Use the pandas.read_html() to extract and convert HTML tables into pandas dataframes
# listofdataframes: a list of dataframes
listofdataframes = pd.read_html(r.text)

/var/folders/ck/nfs6c5bs5sg0lmxls5jsnql80000gn/T/ipykernel_63489/2969171047.py:3: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  listofdataframes = pd.read_html(r.text)


In [6]:
# we are ONLY interested in the second dataframe 
listofdataframes[1].head()

,Country,World Bank[2],Year,Our World in Data[9],Year.1
0,Aruba,NaN,NaN,NaN,NaN
1,Afghanistan,54.5%,2016.0,54.5%,2016.0
2,Angola,32.3%,2018.0,32.3%,2018.0
3,Albania,22%,2020.0,21.8%,2019.0
4,Andorra,NaN,NaN,NaN,NaN


----------------

In [ ]:
# In this extract function, I am only interested in the second table (index = 1)
# You can return the whole list of tables if your ETL requires that.
def extract(url: str) -> pd.DataFrame:
    """This function returns a list of dataframes scraped 
    from the provided web page url

    Args:
        url (str): webpage to be scraped by requests/pandas 

    Returns:
        pd.DataFrame: the first scraped dataframe
    """
    # this scrapes the web page and return a deep copy 
    # of the first returned dataframe
    headers= {
        "User-Agent": "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:47.0) Gecko/20100101 Firefox/47.0"
    }
    r= requests.get(url=url, headers=headers)
    return pd.read_html(r.text)[1].copy()
    

##### Call the function extract() and perform initial exploratory data analysis:
- display the dataframe's information and note the columns' datatypes
- display number of rows and columns
- display the first 10 rows of the dataframe
- display the last 5 rows of the dataframe

In [8]:
# calling the function extract() to return the table as a dataframe
data = extract(url)

/var/folders/ck/nfs6c5bs5sg0lmxls5jsnql80000gn/T/ipykernel_63489/3197759963.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(r.text)[1].copy()


In [9]:
# displaying the information of the dataframe 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 216 entries, 0 to 215
Data columns (total 5 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Country               216 non-null    object 
 1   World Bank[2]         154 non-null    object 
 2   Year                  154 non-null    float64
 3   Our World in Data[9]  156 non-null    object 
 4   Year.1                156 non-null    float64
dtypes: float64(2), object(3)
memory usage: 8.6+ KB


In [10]:
# using .head(10) to display the first 10 rows of the dataframe
data.head(10)

,Country,World Bank[2],Year,Our World in Data[9],Year.1
0,Aruba,NaN,NaN,NaN,NaN
1,Afghanistan,54.5%,2016.0,54.5%,2016.0
2,Angola,32.3%,2018.0,32.3%,2018.0
3,Albania,22%,2020.0,21.8%,2019.0
4,Andorra,NaN,NaN,NaN,NaN
5,United Arab Emirates,NaN,NaN,NaN,NaN
6,Argentina,39.2%,2022.0,42%,2020.0
7,Armenia,26.5%,2021.0,27%,2020.0
8,Antigua and Barbuda,NaN,NaN,NaN,NaN
9,Australia,NaN,NaN,NaN,NaN


In [11]:
# display the last 5 rows of the dataframe
data.head()

,Country,World Bank[2],Year,Our World in Data[9],Year.1
0,Aruba,NaN,NaN,NaN,NaN
1,Afghanistan,54.5%,2016.0,54.5%,2016.0
2,Angola,32.3%,2018.0,32.3%,2018.0
3,Albania,22%,2020.0,21.8%,2019.0
4,Andorra,NaN,NaN,NaN,NaN


# Transform

I will perform the following tasks in order:

1. Split the dataframe (whole dataset) into the two the datasets (dataframes)
    1. World Bank dataset: contains columns (County, World Bank[2], Year). Rename the columns to (`country`, `wb_povertyrate`, `year`). Name the dataframe `wb_data`
    2. Our World in Data dataset: contains columns (County, Our World in Data[9], Year.1). Rename the columns to (`country`, `owid_povertyrate`, `year`). Name the dataframe `owid_data`
2. Convert the poverty rate columns from string to numeric values
    1. `wb_data` dataframe: Convert `wb_povertyrate` datatype to `float`. You should first remove the tainling `%` character. Refer to the in-class lab.
    2. `owid_data` dataframe: Convert `owid_povertyrate` datatype to `float`. You should first remove the tainling `%` character. Refer to the in-class lab.
3. Drop rows that contain missing data from `wb_data` and `owid_data` dataframes.
4. Convert the `year` column's datatype in both tables to `int`.
5. Return the two dataframes `wb_data` and `owid_data` using the statement `return wb_data, owid_data`. The return variable is a tuple with `wb_data` as the first element and `owid_data` as the second.

In [12]:
# a function to convert a string that ends with % to a float
# the string literal must be a float or int value tailed with a %
# for this function to work.
def convert_to_float(x):
    if isinstance(x, str) and x.endswith('%'):
        return float(x[:-1])
    return x

In [ ]:
def transform(data: pd.DataFrame) -> tuple:
    """This function performs a list of transformations.

    Args:
        data (pd.DataFrame): dataframe before transformations

    Returns:
        a tuple of two pd.DataFrame(s): a tuple of transformed data. The first item is the 
        World Bank data and the second item is the Our World in Data  
    """
    # Split the dataframe into two dataframe
    # World Bank dataframe and Our World in Data dataframe

    ## Renaming the columns to the appropriate name
    wb_data = data[["Country", "World Bank[2]", "Year"]].copy()
    wb_data.columns = ["country", "wb_povertyrate", "year"]
    owid_data=data[["Country", "Our World in Data[9]", "Year.1"].copy()]
    owid_data.columns = ["country", "owid_povertyrate", "year"]

    #converted wb_poverty rate & owid_povertyrate from str to float
    # Your code goes here
    wb_data["wb_povertyrate"] = wb_data["wb_povertyrate"].apply(lambda x: convert_to_float(x))
    owid_data['owid_povertyrate'] = owid_data['owid_povertyrate'].apply(lambda x: convert_to_float(x))
    # drop rows that contain missing data
    # Your code goes here
    wb_data.dropna(inplace=True)
    owid_data.dropna(inplace=True)
    # converted year from float to int
    wb_data['year'] = wb_data['year'].astype('Int32')
    owid_data['year'] = owid_data['year'].astype('Int32')

    
    # return the two dataframes
    return wb_data, owid_data

In [ ]:
#extracted the url 
data = extract(url)
#transformed wb_data, owid_data using the data extraction
wb_data, owid_data = transform(data)

/var/folders/ck/nfs6c5bs5sg0lmxls5jsnql80000gn/T/ipykernel_63489/3197759963.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(r.text)[1].copy()
/var/folders/ck/nfs6c5bs5sg0lmxls5jsnql80000gn/T/ipykernel_63489/3240880377.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  owid_data['owid_povertyrate'] = owid_data['owid_povertyrate'].apply(lambda x: convert_to_float(x))
/var/folders/ck/nfs6c5bs5sg0lmxls5jsnql80000gn/T/ipykernel_63489/3240880377.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

In [ ]:
# display the information of wb_data
#displays the general information, type, and non null count of each column
wb_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 154 entries, 1 to 215
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   country         154 non-null    object 
 1   wb_povertyrate  154 non-null    float64
 2   year            154 non-null    Int32  
dtypes: Int32(1), float64(1), object(1)
memory usage: 4.4+ KB


In [ ]:
# display the information of owid_data
#displays the general information, type, and non null count of each column
owid_data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 156 entries, 1 to 215
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   country           156 non-null    object 
 1   owid_povertyrate  156 non-null    float64
 2   year              156 non-null    Int32  
dtypes: Int32(1), float64(1), object(1)
memory usage: 4.4+ KB


In [17]:
# display the first rows of wb_data
# your code goes here 
#displaying the first 5 records for wb_data
wb_data.head()

,country,wb_povertyrate,year
1,Afghanistan,54.5,2016
2,Angola,32.3,2018
3,Albania,22.0,2020
6,Argentina,39.2,2022
7,Armenia,26.5,2021


In [18]:
# display the first rows of owid_data
# your code goes here owid
#displaying the first 5 records for owid_data
owid_data.head()


,country,owid_povertyrate,year
1,Afghanistan,54.5,2016
2,Angola,32.3,2018
3,Albania,21.8,2019
6,Argentina,42.0,2020
7,Armenia,27.0,2020


In [19]:
# display the number of missing data per column in the wb_data daraframe
# (use the functions isna() and sum())
# your code goes here 
#checking for null data to confirm that all na values have been dropped
wb_data.isna().sum()

country           0
wb_povertyrate    0
year              0
dtype: int64

In [20]:
# display the number of missing data per column in the owid_data daraframe
# (use the functions isna() and sum())
# your code goes here 
#checking for null data to confirm that all na values have been dropped
owid_data.isna().sum()


country             0
owid_povertyrate    0
year                0
dtype: int64

# Load

Each of the two dataframes (wb_data and owid_data) should be loaded into a 

1. [10 points] csv file (worldbank_poverty.csv and ourworld_indata_poverty.csv)
2. [15 points] sqlite database named povertystudies and tables named worldbank_poverty and ourworld_indata_poverty. The primary key in both tables is the `country` column

In [21]:
# general parameters
dbname = 'povertystudies'
wb_tablename = 'worldbank_poverty'
owid_tablename = 'ourworldindata_poverty'
# assigning csvfile names
wb_csvfile = 'worldbank_poverty.csv'
owid_csvfile = 'ourworld_indata_poverty.csv'

In [22]:
# load to a csv file
def loadcsv(data: pd.DataFrame, filename: str) -> None:
    """This function loads the argument dataframe data into the csv file identified by filename

    Args:
        data (pd.DataFrame): transformed dataframe
        filename (str): the csv
    """
    # your code goes here
    #This loads the data to the csv filename variable
    data.to_csv(filename,index=False)


In [23]:
# load to a database
def loaddb(data: pd.DataFrame, dbname: str, tablename: str) -> None:
    """This function loads the extracted and transformed data into a sqlite database

    Args:
        data (pd.DataFrame): data to be stored after extract and transform phases
        dbname (str): database name
        tablename (str): table name
    """
    
    # create a connection to the database using sql connection
    # if the database does not exist, it will be created (a .db file in the current folder)
    sqlconnection=sqlite3.connect(dbname)

    # Write the dataframe data into the database
    # if_exists: `replace` if you want to overwrite teh data 
    # OR
    # `append` to add new records
    # remark that I added country as the primary key of the table
    
    # your code goes here
    data.to_sql(tablename, sqlconnection, if_exists='replace', index=False, 
                dtype={'country': 'TEXT PRIMARY KEY'})

### Logging

In [24]:
# [10 points] use the following function to log your data engineering pipeline
def log(logfile: str, message: str) -> None:
    """This function writes the message 'message' and time of occurrence into the file logfile.

    Args:
        logfile (str): log file
        message (str): message to be written into the log file
    """
    # datetime format. Remark that:
    ## .%f appends the parts of teh second
    ## the tailing ', ' are intentionally used to separate the timestamp and the logged message
    datatime_format = '%Y-%m-%d-%H:%M:%S.%f, '
    datetimestamp = datetime.now().strftime(datatime_format)
    
    ## log events are separated by the new character
    with open(logfile, 'a') as file:
        file.write(datetimestamp + message + '\n')

In [25]:
# [20 points] call the extract, transform, and load functions. 
# You will need to call the loadcsv and loaddb twice each to load each of the obtained datasets.
log_file = 'etllog.txt'
#Pipline is starting 
log(log_file, "ETL Pipeline Started")
# Extract phase log has started- extracting url
log(log_file, 'Extract Phase Started')
data = extract(url=url)
log(log_file, 'Extract Phase Ended')
# Transforming phase has started, transforming the two df
log(log_file, 'Transform Phase Started')
wb_data, owid_data = transform(data)
log(log_file, 'Transform Phase Ended')
# Loading phase started, running two loaddb functions, one for each dataframe
log(log_file, 'Load Phase (sqlite db) Started')
loaddb(wb_data, dbname, wb_tablename)
loaddb(owid_data, dbname, owid_tablename)
log(log_file, 'Load Phase (sqlite db) Ended')
# running two loadcsv functions, one for each dataframe
log(log_file, 'Load Phase (csv) Started')
loadcsv(wb_data, wb_csvfile)
loadcsv(owid_data, owid_csvfile)
log(log_file, 'Load Phase (csv) Ended')

/var/folders/ck/nfs6c5bs5sg0lmxls5jsnql80000gn/T/ipykernel_63489/3197759963.py:20: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  return pd.read_html(r.text)[1].copy()
/var/folders/ck/nfs6c5bs5sg0lmxls5jsnql80000gn/T/ipykernel_63489/3240880377.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  owid_data['owid_povertyrate'] = owid_data['owid_povertyrate'].apply(lambda x: convert_to_float(x))
/var/folders/ck/nfs6c5bs5sg0lmxls5jsnql80000gn/T/ipykernel_63489/3240880377.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.o

# Data Analysis of the database content

1. Connect to the database and create a cursor
2. Display the two tables information use the query f'PRAGMA table_info({table_name})'
3. Write 10 different SQL queries. Queries should be different in structure and provide different insight or give different answers.

> `query = f'SELECT year, AVG(wb_povertyrate) FROM {wb_tablename} GROUP BY year'`

> `query = f'SELECT year, AVG(owid_povertyrate) FROM {owid_tablename} GROUP BY year'`

In [ ]:
# Connect the database created in the load stage and create a curser
# connection:

# your code goes here
# Defining connection
connection = sqlite3.connect(dbname)

# A database cursor is used to query a database and fetch results
cur = connection.cursor()

In [ ]:
## returned values for each row:
## {id, name, type, notnull, default_value, primary_key}
# query to return id, name, type,not null, default_value & primary-key for wb
query = f'PRAGMA table_info({wb_tablename})'
# execute the query for wb
result = cur.execute(query)
# fetch all the results using fetchall()
result.fetchall()


[(0, 'country', 'TEXT', 0, None, 1),
 (1, 'wb_povertyrate', 'REAL', 0, None, 0),
 (2, 'year', 'INTEGER', 0, None, 0)]

In [ ]:
## returned values for each row:
## {id, name, type, notnull, default_value, primary_key}
# query to return id, name, type,not null, default_value & primary-key for wb
query = f'PRAGMA table_info({owid_tablename})'
# execute the query for owid
result = cur.execute(query)
# fetch all the results using fetchall()
result.fetchall()
## your code goes here

[(0, 'country', 'TEXT', 0, None, 1),
 (1, 'owid_povertyrate', 'REAL', 0, None, 0),
 (2, 'year', 'INTEGER', 0, None, 0)]

In [ ]:
# create, run and display the results of 10 SQL queries
# This query analyzes how the avg wb_poverty rate changes over years (1)
query = f"""
SELECT year, ROUND(AVG(wb_povertyrate), 2) AS avg_wb_rate
FROM {wb_tablename}
GROUP BY year
ORDER BY year;
"""
result = cur.execute(query)
result.fetchall()

# your code goes here

[(2009, 36.3),
 (2010, 25.75),
 (2011, 30.13),
 (2012, 33.54),
 (2013, 33.48),
 (2014, 48.13),
 (2015, 27.81),
 (2016, 43.03),
 (2017, 27.78),
 (2018, 36.01),
 (2019, 24.24),
 (2020, 16.06),
 (2021, 20.22),
 (2022, 23.3),
 (2023, 14.9),
 (2024, 44.9)]

In [30]:
# This query will show me countries with wb_povertyrate under 2 and the year they achieved that rate (2)
query= f""" 
SELECT country, wb_povertyrate, year
FROM {wb_tablename}
WHERE wb_povertyrate < 2
"""
result = cur.execute(query)
result.fetchall()

[('China', 0.0, 2020), ('Ukraine', 1.6, 2020)]

In [31]:
# This query will show me countries with owid_povertyrate under 2 and the year they achieved that rate (3)
query= f""" 
SELECT country, owid_povertyrate, year
FROM {owid_tablename}
WHERE owid_povertyrate < 2
"""
result= cur.execute(query)
result.fetchall()


[('China', 0.0, 2020), ('Ukraine', 1.4, 2020)]

#### I see a discrepency in between the wb and owid data, specifically in Ukraine, I'm going to now run a query to discover other discrepencies.


In [32]:
# This query will 
# run the poverty rate and look at data where the absolute wb-povertyrate - owid_povertyrate is bigger than 0, 
# which would point out that there is a discrepency between the two datasets (4)
query = f"""
SELECT w.country, w.year, w.wb_povertyrate, o.owid_povertyrate, ROUND(w.wb_povertyrate - o.owid_povertyrate, 2) AS diff
FROM {wb_tablename} w,  {owid_tablename} o
WHERE w.country = o.country AND w.year = o.year
AND ABS(w.wb_povertyrate - o.owid_povertyrate) > 0
ORDER BY ABS(diff) DESC, w.year DESC;
"""
result= cur.execute(query)
result.fetchall()

[('Sao Tome and Principe', 2017, 55.5, 66.7, -11.2),
 ('Ukraine', 2020, 1.6, 1.4, 0.2),
 ('Cabo Verde', 2015, 35.2, 35.0, 0.2)]

In [33]:
# this query checks the wb poverty rate and reported date in morocco (5)
query=f"""
SELECT country, wb_povertyrate, year
FROM {wb_tablename}
WHERE country = "Morocco"
"""
result= cur.execute(query)
result.fetchall()


[('Morocco', 4.8, 2013)]

In [34]:
# This query looks at the number of reported countries per year according to the WorldBank data (6)
query = f"""
SELECT year, COUNT(*) AS countries_reporting
FROM {wb_tablename}
GROUP BY year
ORDER BY year;
"""
result= cur.execute(query)
result.fetchall()

[(2009, 3),
 (2010, 2),
 (2011, 4),
 (2012, 8),
 (2013, 5),
 (2014, 6),
 (2015, 9),
 (2016, 9),
 (2017, 8),
 (2018, 16),
 (2019, 15),
 (2020, 13),
 (2021, 37),
 (2022, 17),
 (2023, 1),
 (2024, 1)]

In [35]:
# this query displays the number of unique country records in the owid table (7)
query= f"""
SELECT COUNT(DISTINCT country) AS num_countries
FROM {owid_tablename};
"""
result= cur.execute(query)
result.fetchall()

[(156,)]

In [36]:
# This query looks at the country with the highest wb_povertyrate (8)
query= f"""
SELECT country, wb_povertyrate
FROM {wb_tablename}
ORDER BY wb_povertyrate DESC
LIMIT 1
"""
result= cur.execute(query)
result.fetchall()

[('South Sudan', 82.3)]

In [37]:
# This query looks at the rounded average owid_povertyrate in 2019- chose that year since that was the year that the COVID lockdown started (9)
query= f"""
SELECT ROUND(AVG(owid_povertyrate)) AS avg_2020_rate, year
FROM {owid_tablename}
WHERE year = 2019;
"""
result= cur.execute(query)
result.fetchall()

[(21.0, 2019)]

In [38]:
# This function checks what 5 countries had the lowest wb_povertyrate in 2019 (10)
query= f"""
SELECT country, wb_povertyrate
FROM {wb_tablename}
WHERE year=2019
ORDER BY wb_povertyrate ASC
LIMIT 5
"""
result= cur.execute(query)
result.fetchall()

[('Maldives', 5.4),
 ('Marshall Islands', 7.2),
 ('Norway', 12.7),
 ('Sri Lanka', 14.3),
 ('Vanuatu', 15.9)]